# Representation of 2-dimensional Ising model on rectangular lattice with periodic boundary conditions

Discussion of model representation and operation that are going to be used in calculations 

### Hamiltonian of the system

Hamiltonian of isotropic Ising model is given as:
$$
    {\cal H}(J,H) = -\frac{J}{2} \sum_{i}\sum_{j\in n(i)}\sigma_i\sigma_j - H \sum_{i}\sigma_i
$$
where summation over $j$ goes over all neighbours of $i$ reprezented by $n(i)$, and $\sigma\in\{\pm1\}$.
In the first example we will be interested in nearest neigbours approximation when $j=i\pm1$

### Modeling with boolean matrix

As spin can take only two values, from computer modeling perspective it might be advantageous to map $\{-1,1\} \rightarrow \{0,1\} = \{\mbox{False}, \mbox{True}\}$ taking advantage of the fact that 0 represents False and 1 any other integer represents True, multiplication of the spins can be represented as negation of exclusive disjunction:

In [1]:
[[not (True ^ True), not (True ^ False)],[not (False ^ True), not (False ^ False)]]

[[True, False], [False, True]]

or negation of substraction:

In [2]:
[[not (True - True), not (True - False)],[not (False - True), not (False - False)]]

[[True, False], [False, True]]

In [3]:
import numpy as np
from math import sqrt

truthnessVector = np.array([True, False])
print('Defined vector: ', truthnessVector)
print('Value of the operation over the vector: ', ~ (truthnessVector ^ truthnessVector))
invertedVector = np.roll(truthnessVector, 1, axis=0)
print('Inverted vector: ', invertedVector)
print('Value of the operation between the vector and its inverse: ', ~ (truthnessVector ^ invertedVector))

ModuleNotFoundError: No module named 'numpy'

That demonstartes the eqivalence between negation of exclusive disjunction and multiplication between Ising spins up to linear transformation. 

### Transformation between canonical Ising model and boolean representation

Summand are in case of canonical Ising model are taking values of $\{-1,1\}$ and in case of boolean representation $\{0,1\}$. That means that linear transformation between them is:
$$
    \mbox{booleanRepresantation}(x) \rightarrow \mbox{canonicalRepresentation}(y): 2 x -1 \\
    \mbox{canonicalRepresentation}(y) \rightarrow \mbox{booleanRepresantation}(x): \frac{y + 1}{2}
$$

In that way, calculations of linear functionals over $\sigma^N$ spaces, as in case of hamiltonian or total magnetization could be easily mapped with the same transformation. 

And taking advantage of Numpy library allows for fast operations such as scalar multiplication or summation:

In [36]:
np.sum(2*[truthnessVector, invertedVector])

4

### Periodic boundary conditions and calculations in isotropic nearest neighbours approximation

Also shifting operation allows for fast calculation of energy of a configuration of a system with periodic boundary conditions by means of:

In [37]:
exampleState = np.array([[True, True, False],[True, False, False], [True, True, True]])
print('Example state: \n', exampleState)

Example state: 
 [[ True  True False]
 [ True False False]
 [ True  True  True]]


In [38]:
print('X-axes shifted state: \n', np.roll(exampleState, 1, axis=1))

X-axes shifted state: 
 [[False  True  True]
 [False  True False]
 [ True  True  True]]


In [39]:
print('Y-axes shifted state: \n', np.roll(exampleState, 1, axis=0))

Y-axes shifted state: 
 [[ True  True  True]
 [ True  True False]
 [ True False False]]


Comment: allowing for next-nearest-neighbour corrections would include just shifts for more then one place (with possibly different interaction constant). Also lifting requiremet for isotropy would be achieved with an introduction of an additional constant.

### Example of calculation of total magnetization $M$ and average magnetization per node $m$

$$
    M = \sum_i \sigma_i
$$

In [40]:
exampleState = np.array([[True, True, False],[True, False, False], [True, True, True]])
print('Example state: \n', exampleState)

Example state: 
 [[ True  True False]
 [ True False False]
 [ True  True  True]]


In [41]:
numberOfNodes = np.size(exampleState)
print('Total number of nodes:', numberOfNodes)
totalMagnetization = 2 * np.sum(exampleState) - numberOfNodes
print('Total magnetization M:', totalMagnetization)
print('Average magnetization per node m:', totalMagnetization/numberOfNodes)

Total number of nodes: 9
Total magnetization M: 3
Average magnetization per node m: 0.333333333333


In [42]:
print(np.average(exampleState), 2*(np.average(exampleState)) - 1)

0.666666666667 0.333333333333


### Example of calculation of energy for PBC in nearest neighbours approx in absence of exprenal H-field

$$
    {\cal H}(J,0) = -\frac{J}{2} \sum_{i}\sum_{j\in n(i)}\sigma_i\sigma_j \\
                  = - J \sum_{i}\sigma_{i}\sigma_{i \mbox{1-shifted_right}}  - J \sum_{i}
                  \sigma_{i}\sigma_{i-\mbox{ 1-shifted_down}}
$$
due to symetry $\sigma_i\sigma_j = \sigma_j\sigma_i$. Taking J=1 for convenience 

In [43]:
exampleState = np.array([[True, True, False],[True, False, False], [True, True, True]])
print('Example state: \n', exampleState)

Example state: 
 [[ True  True False]
 [ True False False]
 [ True  True  True]]


#### Calculation of the right-neighbour terms:

In [44]:
~(exampleState ^ np.roll(exampleState, 1, axis=1))

array([[False,  True, False],
       [False, False,  True],
       [ True,  True,  True]], dtype=bool)

In [45]:
- np.sum(~(exampleState ^ np.roll(exampleState, 1, axis=1)))

-5

#### Calculation of the donw-neighbour terms:

In [46]:
~(exampleState ^ np.roll(exampleState, 1, axis=0))

array([[ True,  True, False],
       [ True, False,  True],
       [ True, False, False]], dtype=bool)

In [47]:
- np.sum(~(exampleState ^ np.roll(exampleState, 1, axis=1)))

-5

#### Total energy  and average energy per node $\epsilon$

Total energy in units of interaction constant $\frac{E}{J}$

In [48]:
E = -np.sum(~(exampleState ^ np.roll(exampleState, 1, axis=1))) - np.sum(~(exampleState ^ np.roll(exampleState, 1, axis=1)))
print('E/J: ', E)

E/J:  -10


Average energy per node $\epsilon$ in the same units
$\frac{\epsilon}{J}$:

In [49]:
print('e/J: ', E/np.size(exampleState))

e/J:  -1.11111111111


Or in canonical Ising representation:

In [59]:
print('e/J: ', 2*(-np.average(~(exampleState ^ np.roll(exampleState, 1, axis=1))) - np.average(~(exampleState ^ np.roll(exampleState, 1, axis=0))))-1)

e/J:  -3.75


In [61]:
print('variance of e/J: ', 4*(np.var(~(exampleState ^ np.roll(exampleState, 1, axis=1))) + np.average(~(exampleState ^ np.roll(exampleState, 1, axis=0)))))

variance of e/J:  3.25


### Standar deviation of average magnetization per node

Variance:
$$
    Var_{m} = \sum_i (\sigma_i - m)^2
$$
and standard deviation:
$$
    SD_{m} = \sqrt{\frac{\sum_i (\sigma_i - m)^2} {N}}
$$

In [50]:
exampleState = np.array([[True, True, False],[True, False, False], [True, True, True]])
numberOfNodes = np.size(exampleState)
print('Example state: \n', exampleState)

Example state: 
 [[ True  True False]
 [ True False False]
 [ True  True  True]]


In [51]:
averageMagnetizationPerNodeBooleanRepresentation = np.sum(exampleState)/numberOfNodes
print('average magnetization per node in boolean representation: ', averageMagnetizationPerNodeBooleanRepresentation)

average magnetization per node in boolean representation:  0.666666666667


In [52]:
print('difference:\n', exampleState - averageMagnetizationPerNodeBooleanRepresentation)
print('square difference:\n', (exampleState - averageMagnetizationPerNodeBooleanRepresentation) ** 2)
print('variance: ', np.sum((exampleState - averageMagnetizationPerNodeBooleanRepresentation)**2)/numberOfNodes)
print('standard deviation: ', sqrt(np.sum((exampleState - averageMagnetizationPerNodeBooleanRepresentation)**2)/numberOfNodes))
print('Using Numpy functions | average: ', np.average(exampleState))
print('Using Numpy functions | variance: ', np.var(exampleState))
print('Using Numpy functions | standard deviation: ', sqrt(np.var(exampleState)))

difference:
 [[ 0.33333333  0.33333333 -0.66666667]
 [ 0.33333333 -0.66666667 -0.66666667]
 [ 0.33333333  0.33333333  0.33333333]]
square difference:
 [[ 0.11111111  0.11111111  0.44444444]
 [ 0.11111111  0.44444444  0.44444444]
 [ 0.11111111  0.11111111  0.11111111]]
variance:  0.222222222222
standard deviation:  0.4714045207910317
Using Numpy functions | average:  0.666666666667
Using Numpy functions | variance:  0.222222222222
Using Numpy functions | standard deviation:  0.4714045207910317


Accordin to [Expectation and Variance of lineary transformed argument](/Expectation%20and%20Variance%20of%20lineary%20transformed%20argument.ipynb) values for canonical Ising model are:

In [53]:
print('Using Numpy functions | average: ', 2*np.average(exampleState)-1)
print('Using Numpy functions | variance: ', 4*np.var(exampleState))
print('Using Numpy functions | standard deviation: ', 2*sqrt(np.var(exampleState)))

Using Numpy functions | average:  0.333333333333
Using Numpy functions | variance:  0.888888888889
Using Numpy functions | standard deviation:  0.9428090415820634


### Calculating energy differnce for a single spin flip

Let's say that we are interested in spin at position <2,3>. The whole lattice can be easily sliced as:

In [54]:
exampleState = np.array([[False, False, False, False],[False, False, False, False], [True, True, False, False],[False, False, True, False]])
numberOfNodes = np.size(exampleState)
print('Example state: \n', exampleState)

Example state: 
 [[False False False False]
 [False False False False]
 [ True  True False False]
 [False False  True False]]


In [55]:
i = 2; j = 4;
relevantSlice = np.pad(exampleState, 1, mode='wrap')[i-1:i+2,j-1:j+2]
print(relevantSlice)
E1 = -np.sum(~(relevantSlice ^ np.roll(relevantSlice, 1, axis=1))) - np.sum(~(relevantSlice ^ np.roll(relevantSlice, 1, axis=1)))
print('E1: ', E1)
relevantSlice[1,1] = ~relevantSlice[1,1]
E2 = -np.sum(~(relevantSlice ^ np.roll(relevantSlice, 1, axis=1))) - np.sum(~(relevantSlice ^ np.roll(relevantSlice, 1, axis=1)))
print('E2: ', E2)
print('Difference E2-E1: ', E2-E1)

[[False False False]
 [False False False]
 [False False  True]]
E1:  -14
E2:  -10
Difference E2-E1:  4


What means that the filpped state has higher energy